# Traffic sign recognition using Tensorflow 

For training I am using German sign dataset which can be found here: http://benchmark.ini.rub.de/

In [ ]:
import os
import csv
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
from keras.preprocessing.image import ImageDataGenerator
import cv2

Setting path to train and test data.

In [ ]:
ROOT_PATH = "./datasets"
TRAIN_DATA_DIR = os.path.join(ROOT_PATH, "training_images/Final_Training/Images/")
TEST_DATA_DIR = os.path.join(ROOT_PATH, "testing_dataset/Final_Test/Images/")
IMAGE_WIDTH = 32
IMAGE_LENGTH = 32
BATCH_SIZE = 128

Loading train data.

In [ ]:
def load_train_data(data_dir):
    directories = [d for d in os.listdir(data_dir)
                  if os.path.isdir(os.path.join(data_dir, d))]
    labels = []
    images = []
    for d in directories:
        path = data_dir + d + '/'
        csv_file = open(path + 'GT-' + d + '.csv')
        temp_labels, temp_images = get_data(csv_file, path)
        labels += temp_labels
        images += temp_images
        
    return labels, images

Loading test data.

In [ ]:
def load_test_data(data_dir):
    csv_file = open(data_dir + 'GT-final_test.csv')
        
    return get_data(csv_file, data_dir)

In [ ]:
def get_data(csv_file, path):
    images = []
    labels = []
    reader = csv.reader(csv_file, delimiter=';')
    reader.next()
    for row in reader:
        images.append(skimage.transform.resize(plt.imread(path + row[0]), (IMAGE_WIDTH, IMAGE_LENGTH)))
        labels.append(int(row[7]))
    csv_file.close()
    return labels, images

In [ ]:
def generate_blur_image(img):
    size = 2
    kernel_motion_blur = np.zeros((size, size))
    kernel_motion_blur[int((size-1)/2), :] = np.ones(size)
    kernel_motion_blur = kernel_motion_blur / size
    return cv2.filter2D(img, -1, kernel_motion_blur)

In [ ]:
train_labels, train_images = load_train_data(TRAIN_DATA_DIR)
val_labels, val_images = load_test_data(TEST_DATA_DIR)

In [ ]:
train_labels_arr = np.array(train_labels)
train_images_arr = np.array(train_images)

In [ ]:
val_labels_arr = np.array(val_labels)
val_images_arr = np.array(val_images)

In [ ]:
datagen = ImageDataGenerator(rotation_range=17,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=False,
                             fill_mode='nearest')

In [ ]:
for temp_images, temp_labels in datagen.flow(train_images_arr, train_labels_arr,
                                             batch_size=len(train_labels), shuffle=False):
    break

In [ ]:
train_images_arr = np.concatenate((train_images_arr, temp_images))

In [ ]:
train_labels_arr = np.concatenate((train_labels_arr, temp_labels))

In [ ]:
blur_images = []
blur_labels = []

images_count = len(train_labels)

for i in range(images_count):
    blur_images.append(generate_blur_image(train_images[i]))
    blur_labels.append(train_labels[i])

In [ ]:
train_labels_arr = np.concatenate((train_labels_arr, np.array(blur_labels)))
train_images_arr = np.concatenate((train_images_arr, np.array(blur_images)))

Creating bar chart for dataset Visualization.

In [ ]:
data = {x:train_labels.count(x) for x in train_labels}.values()

plt.figure(figsize=(20, 10))

y_pos = np.arange(len(data))

plt.bar(y_pos, data, align='center', alpha = 0.5)
plt.xticks(y_pos, y_pos)
plt.axis('tight')
plt.ylabel('Number of examples')
plt.title('Training dataset')
plt.show()

In [ ]:
print(train_images_arr.shape)

In [ ]:
def conv2d(input_layer, filters, padding):
    return tf.layers.conv2d(
        inputs=input_layer,
        filters=filters,
        kernel_size=(5, 5),
        padding=padding,
        activation=tf.nn.elu)

def pooling(input_layer):
    return tf.layers.max_pooling2d(
        inputs=input_layer, 
        pool_size=(2, 2), 
        strides=(2, 2),
        padding='same')

with tf.device('/device:GPU:0'):    
    images = tf.placeholder(tf.float32, [None, IMAGE_WIDTH, IMAGE_LENGTH, 3], name = "images")
    labels = tf.placeholder(tf.int64, [None])
    
    conv1 = conv2d(images, 12, 'valid')
    pool1 = pooling(conv1)
    
    dropout1 = tf.layers.dropout(inputs=pool1, rate=0.9)
    conv2 = conv2d(dropout1, 24, 'valid')
    pool2 = pooling(conv2)
    
    dropout2 = tf.layers.dropout(inputs=pool2, rate=0.7)
    #conv3 = conv2d(dropout2, 32, 'same')
    #pool3 = pooling(conv3)
    #dropout3 = tf.layers.dropout(inputs=pool3, rate=0.6)
   
    flat = tf.layers.flatten(dropout2)
    f1 = tf.layers.dense(flat, 300, tf.nn.elu)
    
    #dropout = tf.layers.dropout(inputs=flat, rate=0.5)
    
    logits = tf.layers.dense(f1, len(set(train_labels)), tf.nn.elu)

    predicted = tf.nn.softmax(logits)
    predicted_labels = tf.argmax(predicted, 1, name="prediction")
   
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits,
                                                                   labels = labels)
    loss = tf.reduce_mean(cross_entropy)

    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

    correct_prediction = tf.equal(predicted_labels, labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    init = tf.global_variables_initializer()

In [ ]:
saver = tf.train.Saver()
session = tf.Session()
session.run(init)

In [ ]:
def validation(val_images, val_labels):
    val_acc = session.run(accuracy,
                    {images: val_images,
                    labels: val_labels})
    
    return val_acc

In [ ]:
def training_log(steps, loss, val_acc, training_time):
    print('Steps: {}'.format(steps))
    print('Loss: {0} Validation accuracy: {1:.2f}%'
          .format(loss, val_acc * 100))
    minutes, seconds = divmod(training_time, 60)
    hours, minutes = divmod(minutes, 60)
    print('Traning time: {0:02.0f}:{1:02.0f}:{2:05.2f}\n'.format(hours, minutes, seconds))  

In [ ]:
def train_model(training_images, training_labels, val_images, val_labels, steps):
    t_start = time.time()
    prev_val_acc = 0
    for i in range(steps + 1):
        indexes = np.random.choice(np.arange(len(train_images)), BATCH_SIZE, replace = False)
        batch_images = train_images_arr[indexes]
        batch_labels = train_labels_arr[indexes]
        loss_value = session.run([optimizer, loss],
                                 {images: batch_images,
                                 labels: batch_labels})[1]
        if (i % 50 == 0):
            val_acc = validation(val_images, val_labels)
            training_log(i, loss_value, val_acc, time.time() - t_start)
            
            if (prev_val_acc < val_acc and i % 500 == 0):
                prev_val_acc = val_acc
                saver.save(session, './model/model')
        
                
train_model(train_images, train_labels, val_images_arr, val_labels_arr, 10000)

In [ ]:
session.close()